In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("MultilayerPerceptronClassifier").getOrCreate()

23/12/27 15:41:35 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/27 15:41:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 15:41:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
iris = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/iris.csv", header=True, inferSchema=True,
                      sep=",")
iris.show()

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|
|        5.4|       3.7|        1.5|       0.2|Iris-setosa|
|        4.8|       3.4|        1.6|       0.2|Iris-setosa|
|        4.8|       3.0|        1.4|       0.1|Iris-setosa|
|        4.3|       3.0|        1.1|    

In [3]:
from pyspark.ml.clustering import KMeans

In [4]:
from pyspark.ml.feature import VectorAssembler

asb = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="features")

irisas = asb.transform(iris)
irisas.show()

+-----------+----------+-----------+----------+-----------+-----------------+
|sepallength|sepalwidth|petallength|petalwidth|      class|         features|
+-----------+----------+-----------+----------+-----------+-----------------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|[4.4,2.9,1.4,0.2]|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|[4.9,

In [5]:
from pyspark.ml.feature import StringIndexer

ind = StringIndexer(inputCol="class", outputCol="classtrans")

irisas = ind.fit(irisas).transform(irisas)
irisas.show()

+-----------+----------+-----------+----------+-----------+-----------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|         features|classtrans|
+-----------+----------+-----------+----------+-----------+-----------------+----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|       0.0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|       0.0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|       0.0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|       0.0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|       0.0|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|       0.0|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|       0.0|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|       0.0|
|        4.4|       2

In [6]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

irisas = irisas.withColumn("classtrans", irisas["classtrans"].cast(IntegerType()))
irisas.show(5)

+-----------+----------+-----------+----------+-----------+-----------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|         features|classtrans|
+-----------+----------+-----------+----------+-----------+-----------------+----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|         0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|         0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|         0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|         0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|         0|
+-----------+----------+-----------+----------+-----------+-----------------+----------+
only showing top 5 rows



In [7]:
km = KMeans(predictionCol="grupo", maxIter=100, k=3, featuresCol="features")
modelo = km.fit(irisas)

In [8]:
grupos = modelo.transform(irisas)
grupos.show()

+-----------+----------+-----------+----------+-----------+-----------------+----------+-----+
|sepallength|sepalwidth|petallength|petalwidth|      class|         features|classtrans|grupo|
+-----------+----------+-----------+----------+-----------+-----------------+----------+-----+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|         0|    1|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|         0|    1|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|         0|    1|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|         0|    1|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|         0|    1|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|         0|    1|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|         0|    1|
|        5.0|       3.4|        1.5|       0.2|Iri

In [9]:
classe = grupos.select("classtrans").collect()
agrupado = grupos.select("grupo").collect()

from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(classe, agrupado)

print(cm)

[[ 0 50  0]
 [48  0  2]
 [14  0 36]]


In [10]:
acuracia = (cm[0, 1] + cm[1, 0] + cm[2,2]) / 150 # 150 é o total de registros

print(f"Acurácia: {acuracia}")

Acurácia: 0.8933333333333333
